In [68]:
import joblib
import numpy as np
import pandas as pd

from haversine import haversine, Unit, inverse_haversine
from shapely.geometry import Point, LineString

# Load Pickled Model and Scaler

In [69]:
cam_meta = pd.read_csv("data/cam_meta_05302022.csv")
model = joblib.load('locating_angle_model.gz')
scaler = joblib.load('locating_scaler.gz')

In [70]:
def find_slope(angle, cl_angle, direction, xpix, xres):
    if direction == 'n':
        if xpix < (xres/2):
            return np.tan((np.pi/2)+angle+cl_angle)
        else:
            return np.tan((np.pi/2)-angle+cl_angle)
    if direction == 'e':
        if xpix < (xres/2):
            return np.tan(angle+cl_angle)
        else:
            return np.tan(-angle+cl_angle)
    if direction == 's':
        if xpix < (xres/2):
            return np.tan((3*np.pi/2)+angle+cl_angle)
        else:
            return np.tan((3*np.pi/2)-angle+cl_angle)
    if direction == 'w':
        if xpix < (xres/2):
            return np.tan(np.pi+angle+cl_angle)
        else:
            return np.tan(np.pi-angle+cl_angle)

In [79]:
class camera_view:
    def __init__(self, station, direction, tile_num, processed_xres=1856, processed_yres = 1040, precrop_yres = 1392, overlap_size=20, tile_size = [224,224]):
        self.station = station
        self.direction = direction
        self.tile_num = tile_num
        self.processed_xres = processed_xres
        self.processed_yres = processed_yres
        self.precrop_yres = precrop_yres
        self.overlap_size = overlap_size
        self.tile_size = tile_size
        self.full_xres = cam_meta['x_resolution'].loc[(cam_meta['station'] == station) & (cam_meta['dir'] == direction)].iloc[0]#get from metadata
        self.full_yres = cam_meta['y_resolution'].loc[(cam_meta['station'] == station) & (cam_meta['dir'] == direction)].iloc[0]#get from metadata
        self.elevation = cam_meta['elevation'].loc[(cam_meta['station'] == station) & (cam_meta['dir'] == direction)].iloc[0]
        self.cl_angle = cam_meta['center_angle'].loc[(cam_meta['station'] == station) & (cam_meta['dir'] == direction)].iloc[0]
        self.cam_lat = cam_meta['cam_lat'].loc[(cam_meta['station'] == station) & (cam_meta['dir'] == direction)].iloc[0]
        self.cam_long = cam_meta['cam_long'].loc[(cam_meta['station'] == station) & (cam_meta['dir'] == direction)].iloc[0]
        self.pix_centers = self.find_tile_centers()
        self.angle = self.find_angle()
        self.converted_angle = self.convert_angle()
                
    def find_tile_centers(self):
        num_tiles_x = int((self.processed_xres - self.overlap_size)/(self.tile_size[0] - self.overlap_size))
        num_tiles_y = int((self.processed_yres - self.overlap_size)/(self.tile_size[0] - self.overlap_size))
        full_y_crop = (self.precrop_yres / self.processed_yres)
        xres_ratio = self.full_xres/self.processed_xres
        yres_ratio = (self.full_yres*(self.processed_yres/self.precrop_yres))/self.processed_yres
        stepsize_x = (self.tile_size[0]*xres_ratio) - (self.overlap_size * xres_ratio)
        stepsize_y = (self.tile_size[1]*yres_ratio) - (self.overlap_size * yres_ratio)
        x_pix = [round(((self.tile_size[0]*xres_ratio)/2) + (i*stepsize_x)) for i in range(num_tiles_x)]
        y_pix = [round(((self.tile_size[1]*yres_ratio)/2) + (i*stepsize_y) + (((self.precrop_yres-self.processed_yres) / self.precrop_yres)*self.full_yres)) for i in range(num_tiles_y)]
        
        return [[x_pix[j],y_pix[i]] for i in range(len(y_pix)) for j in range(len(x_pix))]
    
    def find_angle(self):
        x_ratio = abs(self.pix_centers[self.tile_num][0]-(self.full_xres/2)) / (self.full_xres/2)
        y_ratio = abs(self.pix_centers[self.tile_num][1] -self.full_yres)/ self.full_yres
        ang1 = model.predict(scaler.transform([[x_ratio, y_ratio, self.elevation]]))
        return ang1
    
    def convert_angle(self):
        dir_dict = {'n': np.pi/2, 'w': np.pi, 's': 3*np.pi/2, 'e': 0}
        if (self.pix_centers[self.tile_num][0]) <= (self.full_xres/2):
            return  self.cl_angle + dir_dict[self.direction] + self.angle
        else:
            return  self.cl_angle + dir_dict[self.direction] - self.angle
        
    def single_camera(self):
        cam_coord = (self.cam_lat, self.cam_long)
        cam_end = inverse_haversine(cam_coord, 30, self.angle)
        left_start = inverse_haversine(cam_coord, 3, self.angle+(np.pi/2))
        left_end = inverse_haversine(left_start, 30, self.angle)
        right_start = inverse_haversine(cam_coord, 3, self.angle-(np.pi/2))
        right_end = inverse_haversine(right_start, 30, self.angle)

        line_left = LineString([cam_coord, cam_end])
        line_center = LineString([left_start, left_end])
        line_right = LineString([right_start, right_end])

        return gpd.GeoDataFrame([line_left, line_center, line_right],  columns=['LineString_obj'],  geometry='LineString_obj')

# Create Camera View Objects

In [72]:
cv1 = camera_view(station='bh', direction='e', tile_num=11)
cv2 = camera_view(station='hp', direction='w', tile_num=8)

cv_list = [cv1, cv2]

# Check for Intersections

In [73]:
df_intersections = pd.read_csv("data/intersections.csv")

In [74]:
df_filt = df_intersections[df_intersections[['station', 'dir']].apply(tuple, axis=1).isin([(cv.station, cv.direction) for cv in cv_list])]
pairs = []
for i, base_cv in enumerate(cv_list):
    for cv in cv_list:
        compare_cam = str((cv.station, cv.direction))
        if compare_cam in df_filt['intersections'].iloc[i]:
            pairs.append({base_cv, cv})
            
tri_pairs = [i for n, i in enumerate(pairs) if i not in pairs[:n]]

# Perform Triangulation

In [75]:
for pair in tri_pairs:
    lpair = list(pair)
    lc1= lpair[0]
    lc2 = lpair[1]
    
    m1 = find_slope(lc1.angle, lc1.cl_angle, lc1.direction, lc1.pix_centers[lc1.tile_num][0], lc1.full_xres)
    b1 = lc1.cam_lat - (m1 * lc1.cam_long)
    m2 = find_slope(lc2.angle, lc2.cl_angle, lc2.direction, lc2.pix_centers[lc2.tile_num][0], lc2.full_xres)
    b2 = lc2.cam_lat - (m2 * lc2.cam_long)

    x_coord = (b2 - b1)/(m1-m2)
    y_coord = (m1 * x_coord) + b1

    print("Fire Estimate between {} and {} Stations: {}".format(lc1.station, lc2.station, (x_coord[0], y_coord[0])))

Fire Estimate between bh and hp Stations: (-116.82696798112472, 33.3529411599077)


In [76]:
df = pd.read_csv("data/landmarks_processed.csv")
df_intersections = df.loc[df['intersection'] == 1].reset_index(drop=True)
test_lms = set(df_intersections['landmark'])
df_intersections

,landmark,station,dir,lat,long,x_pixel,y_pixel,x_res,y_res,intersection,distance,tile
0,otay resevoir - north tip,om,w,32.628680,-116.926238,2315,1153,3072,2048,1,8.518187,24
1,otay resevoir - north tip,sm,s,32.628680,-116.926238,1410,1250,3072,2048,1,7.641891,22
2,mt laguna water tower,ml,s,32.875975,-116.414988,765,637,2048,1536,1,1.892158,12
3,nucal egg ranch,mg,s,33.072523,-116.776892,1731,1117,3072,2048,1,13.023885,14
4,nucal egg ranch,bm,s,33.072523,-116.776892,1067,1127,3072,2048,1,10.144026,12
5,Palomar Observatory right,hp,w,33.348388,-116.859694,161,693,2048,1536,1,2.720259,9
6,Palomar Observatory right,bh,e,33.348388,-116.859694,1044,1011,3072,2048,1,5.747043,12
7,target - san clemente,sclm,n,33.464032,-117.605505,642,1183,3072,2048,1,3.704763,19
8,target - san clemente,sjh,s,33.464032,-117.605505,1681,1152,3072,2048,1,4.332222,22
9,santa ynez reservoir,dwpgm,s,34.072078,-118.567900,2738,1321,3072,2048,1,2.375986,26


In [80]:
for lm in test_lms:
    
    data1 = df_intersections.loc[df_intersections['landmark'] == lm].iloc[0]
    data2 = df_intersections.loc[df_intersections['landmark'] == lm].iloc[1]

    lc1 = camera_view(station=data1['station'], direction=data1['dir'], tile_num=data1['tile'])
    lc2 = camera_view(station=data2['station'], direction=data2['dir'], tile_num=data2['tile'])
        
    print(lc1.station, lc1.angle, (lc1.pix_centers[lc1.tile_num][0], lc1.pix_centers[lc1.tile_num][1]))
    print(lc2.station, lc2.angle, (lc2.pix_centers[lc2.tile_num][0], lc2.pix_centers[lc2.tile_num][1]))
    
    m1 = find_slope(lc1.angle, lc1.cl_angle, lc1.direction, lc1.pix_centers[lc1.tile_num][0], lc1.full_xres)
    b1 = lc1.cam_lat - (m1 * lc1.cam_long)
    m2 = find_slope(lc2.angle, lc2.cl_angle, lc2.direction, lc2.pix_centers[lc2.tile_num][0], lc2.full_xres)
    b2 = lc2.cam_lat - (m2 * lc2.cam_long)
    
    x_coord = (b2 - b1)/(m1-m2)
    y_coord = (m1 * x_coord) + b1
    dist_acc = haversine((data1['long'], data1['lat']), (x_coord, y_coord), unit=Unit.MILES)
    print(lm)
    print("X: {}, Y: {}".format(x_coord, y_coord))
    print("Distance From Actual: ", dist_acc)
    print("\n")

om [0.37028087] (2211, 1283)
sm [0.00051224] (1536, 1283)
otay resevoir - north tip
X: [-116.93436269], Y: [32.63065017]
Distance From Actual:  0.5647380536305667


dwpgm [0.59182953] (523, 1283)
69bravo [0.38734932] (2211, 1283)
santa monica pier
X: [-118.51014109], Y: [34.01046701]
Distance From Actual:  0.7238066933733769


dwpgm [0.7849832] (2887, 1283)
dwpgm [0.7849832] (185, 1283)
santa ynez reservoir
X: [-118.5487598], Y: [34.08640497]
Distance From Actual:  1.4045846090882708


ch [0.40119286] (861, 1283)
rdd [0.02701844] (1536, 983)
palomar medical center
X: [-117.12030447], Y: [33.12426061]
Distance From Actual:  0.10272715071322742


ch [0.01545697] (1536, 1283)
rdd [0.4114182] (2211, 1283)
veritiv distribution warehouse
X: [-117.11413093], Y: [33.10980534]
Distance From Actual:  0.11198831326265313


sclm [0.60583038] (523, 1283)
sjh [0.02211131] (1536, 1283)
target - san clemente
X: [-117.60187204], Y: [33.45042894]
Distance From Actual:  0.5026770677226936


hp [0.7162170